<a href="https://colab.research.google.com/github/yuyu990116/transformers_tutorials/blob/main/P2_mlm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/nlp")

Mounted at /content/drive


In [ ]:
#掩码模型
!pip install datasets
! pip install -U accelerate
! pip install -U transformers
from transformers import AutoTokenizer,AutoModelForMaskedLM,DataCollatorForLanguageModeling,TrainingArguments,Trainer,pipeline
from datasets import Dataset,load_dataset
datasets = load_dataset("pleisto/wikipedia-cn-20230720-filtered",cache_dir="./")
model = AutoModelForMaskedLM.from_pretrained("hfl/chinese-macbert-base")
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")

In [3]:
tokenizer.mask_token,tokenizer.mask_token_id

('[MASK]', 103)

In [4]:
datasets

DatasetDict({
    train: Dataset({
        features: ['completion', 'source'],
        num_rows: 254547
    })
})

In [5]:
from datasets import DatasetDict
train_percentage = 0.8
validation_percentage = 0.1
test_percentage = 0.1

num_rows = datasets['train'].num_rows
train_size = int(num_rows * train_percentage)
validation_size = int(num_rows * validation_percentage)

# Splitting the dataset
splits = datasets['train'].train_test_split(test_size=test_percentage, shuffle=True, seed=42)
train_dataset = splits['train']
validation_dataset = splits['test']

# Additional splitting for validation set
validation_splits = validation_dataset.train_test_split(test_size=validation_percentage, shuffle=True, seed=42)
validation_dataset = validation_splits['train']
test_dataset = validation_splits['test']

# Create a DatasetDict
new_dataset = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset,
    'test': test_dataset,
})

In [6]:
train_datasets=new_dataset["train"]
train_datasets

Dataset({
    features: ['completion', 'source'],
    num_rows: 229092
})

In [7]:
train_datasets.column_names

['completion', 'source']

In [8]:
def data_collator(example):
  return tokenizer(example["completion"],max_length=64,truncation=True,return_special_tokens_mask=True)
tokenized_dataset = new_dataset.map(data_collator,batched=True,remove_columns=train_dataset.column_names)
#bert中的special token有 [cls],[sep],[unk],[pad],[mask]

Map:   0%|          | 0/229092 [00:00<?, ? examples/s]

Map:   0%|          | 0/22909 [00:00<?, ? examples/s]

Map:   0%|          | 0/2546 [00:00<?, ? examples/s]

In [9]:
tokenized_dataset["train"][0].keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'])

In [ ]:
tokenized_dataset["train"][0]

In [14]:
args=TrainingArguments(
    output_dir="./masked_lm",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=64,
    logging_steps=10,
    num_train_epochs=1,
    #load_best_model_at_end=True 这里不能用，好像是因为Evaluation strategy默认为no，既然没有eval函数和标准，也就没必要用这个
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm=True, mlm_probability=0.15)
)
#训练集用于训练模型的参数，而验证集用于评估模型的性能。
#在每个训练周期结束时，模型的性能在验证集上进行评估，以检查模型是否过拟合或者对新数据的泛化能力如何。
#如果没有eval函数，那么或许就根本不需要validation set？
trainer.train()

In [18]:
model=AutoModelForMaskedLM.from_pretrained('/content/drive/MyDrive/nlp/masked_lm/checkpoint-2000')

In [22]:
pipe = pipeline("fill-mask", model=model, tokenizer=tokenizer,device=0,top_k=1)
res=pipe("西安交通[MASK][MASK]博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的博物馆")

In [26]:
res

[[{'score': 0.9988762736320496,
   'token': 1920,
   'token_str': '大',
   'sequence': "[CLS] 西 安 交 通 大 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"}],
 [{'score': 0.9977860450744629,
   'token': 2110,
   'token_str': '学',
   'sequence': "[CLS] 西 安 交 通 [MASK] 学 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"}]]

In [30]:
res[0][0]

{'score': 0.9988762736320496,
 'token': 1920,
 'token_str': '大',
 'sequence': "[CLS] 西 安 交 通 大 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"}

In [37]:
completed_sentence = "西安交通[MASK][MASK]博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的博物馆".replace("[MASK]", "{}").format(*(result[0]["token_str"] for result in res))
# * 是解包运算符，它用于将一个可迭代对象（如元组或列表）的元素解包到函数的参数中。使用 * 来展开生成器表达式的结果。不然replace不接受生成器，会报错。

In [38]:
completed_sentence

"西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的博物馆"

In [41]:
pipe = pipeline("fill-mask", model=model, tokenizer=tokenizer,device=0,top_k=1)
s = "我是一名人民[MASK][MASK]"
res=pipe(s)
output=s.replace("[MASK]","{}").format(*(r[0]["token_str"]for r in res))

In [42]:
output

'我是一名人民党。'